In [ ]:
# Load preprocessed datasets



import pandas as pd

data_path = "../data/"
X_train = pd.read_csv(data_path + "X_train.csv")
X_val = pd.read_csv(data_path + "X_val.csv")
y_train = pd.read_csv(data_path + "y_train.csv").squeeze()  # Convert to Series
y_val = pd.read_csv(data_path + "y_val.csv").squeeze()  # Convert to Series
X_test = pd.read_csv(data_path + "X_test.csv")
test_merged.to_csv("../data/test_merged.csv", index=False)



In [ ]:
# Recreate test_merged
test_merged = datasets['test'].merge(
    datasets['meal_info'], on='meal_id', how='left'
).merge(
    datasets['fulfilment_center_info'], on='center_id', how='left'
)

# Add derived features
test_merged['price_diff'] = test_merged['base_price'] - test_merged['checkout_price']
test_merged['promotion'] = test_merged['emailer_for_promotion'] + test_merged['homepage_featured']

# Encode categorical variables
from sklearn.preprocessing import LabelEncoder
for col in ['category', 'cuisine', 'center_type']:
    le = LabelEncoder()
    test_merged[col] = le.fit_transform(test_merged[col])

In [ ]:
# Define feature columns for test data
feature_cols = [
    "checkout_price", "base_price", "price_diff", "promotion", 
    "category", "cuisine", "city_code", "region_code", "center_type", "op_area"
]

# Prepare X_test
X_test = test_merged[feature_cols]

# Preview X_test
print("X_test - Overview:")
print(X_test.head())


In [10]:
from sklearn.linear_model import LinearRegression
from sklearn.metrics import mean_absolute_error, mean_squared_error, r2_score
import numpy as np

# Initialize the model
model = LinearRegression()

# Train the model
model.fit(X_train, y_train)

# Predict on validation set
y_val_pred = model.predict(X_val)

# Evaluate the model
mae = mean_absolute_error(y_val, y_val_pred)
rmse = np.sqrt(mean_squared_error(y_val, y_val_pred))
r2 = r2_score(y_val, y_val_pred)

print(f"Model Evaluation on Validation Set:")
print(f"Mean Absolute Error (MAE): {mae}")
print(f"Root Mean Squared Error (RMSE): {rmse}")
print(f"R-squared (R2): {r2}")

# Predict on test set
y_test_pred = model.predict(X_test)

# Save test predictions
test_predictions = test_merged[["id"]].copy()
test_predictions["num_orders"] = y_test_pred
test_predictions.to_csv("../results/test_predictions.csv", index=False)

print("\nTest predictions saved to ../results/test_predictions.csv")


Model Evaluation on Validation Set:
Mean Absolute Error (MAE): 194.35814611444079
Root Mean Squared Error (RMSE): 339.2426479106249
R-squared (R2): 0.24550610267294026


NameError: name 'test_merged' is not defined